In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as spstats

%matplotlib inline

In [93]:
def whole_feature_process(df):
    inuteis = ['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin']
    ordinais = ['Pclass', 'Fare_binning']
    categoricas = ['Sex', 'Embarked', 'binAge', 'Alone', 'Title']
    entra_assim_msm = ['Age', 'Relatives', 'Fare', 'Fare_std', 'Age_std']
    #nao sei como generalizar isso
    y = ['Survived']
    
    fm = clean_stuff(df)
    fm = make_alone(fm)
    fm = make_binAge(fm)
    fm = make_fare_binning(fm)
    fm['Relatives'] = fm['Parch'] + fm['SibSp']
    fm = get_col_title(fm)
    fm = get_std_stuff(fm, 'Fare')
    fm = get_std_stuff(fm, 'Age')
    
    fm = fm.drop(inuteis, axis=1)

    fm = pd.get_dummies(fm, columns=categoricas, drop_first=True)
    #soh enquanto eu nao sei como usar ordinais
    fm = pd.get_dummies(fm, columns=ordinais, drop_first=True)
    
    fm['Fare_std'] = fm['Fare_std'].apply(lambda x: abs(x)) 
    fm['Age_std'] = fm['Fare_std'].apply(lambda x: abs(x)) 
    
    return fm

In [94]:
def make_fare_binning(df):
    mask = df['Fare'] > 200

    Fare_binning = df['Fare'] / 200
    df['Fare_binning'] = np.array(np.round(Fare_binning * 10),
                                    dtype='int')
    df.loc[mask, 'Fare_binning'] = 10 
    
    return df

In [95]:
def make_binAge(df):

    i = 7
    arbitrary_jump = 7
    df['binAge'] = 0 # inicializa a coluna

    while i < df['Age'].max():
        mask = df['Age'] > i
        df.loc[mask, 'binAge'] += 1
        i += arbitrary_jump
        
    return df

In [96]:
def make_alone(df):
    df['Alone'] = df.SibSp + df.Parch
    df['Alone'].loc[df['Alone'] > 0] = 0
    df['Alone'].loc[df['Alone'] == 0] = 1
    return df

In [97]:
def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'

In [98]:
def replace_titles(x):
    title = x['Title']
    if title in ['Capt', 'Col', 'Don', 'Jonkheer', 'Major', 'Rev', 'Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

In [99]:
def get_col_title(df):
    df['Title'] = df['Name'].map(lambda x: get_title(x))

    # And replace the titles, so the are normalized to 'Mr', 'Miss' and 'Mrs'
    df['Title'] = df.apply(replace_titles, axis=1)
    
    return df

In [100]:
# pra pegar a quantas std alguem ta da distribuição:
def get_std_stuff(df, col):
    mean = df[col].mean()
    std = np.std(df[col])
    new_name = f"{col}_std"
    df[new_name] = (df[col] - mean) / std
    return df

In [101]:
def clean_stuff(data):
    data.Age.fillna(value=data.Age.mean(), inplace=True)
    data.Fare.fillna(value=data.Fare.mean(), inplace=True)
    data.Embarked.fillna(value=(data.Embarked.value_counts().idxmax()), inplace=True)
    return data

In [102]:
test = pd.read_csv('../data/test.csv')

In [103]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [104]:
fm_matrix = whole_feature_process(test)

In [105]:
fm_matrix.head()

,Age,Fare,Relatives,Fare_std,Age_std,Sex_male,Embarked_Q,Embarked_S,binAge_1,binAge_2,...,Pclass_2,Pclass_3,Fare_binning_1,Fare_binning_2,Fare_binning_3,Fare_binning_4,Fare_binning_5,Fare_binning_7,Fare_binning_8,Fare_binning_10
0,34.5,7.8292,0,0.498407,0.498407,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,47.0,7.0000,1,0.513274,0.513274,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,62.0,9.6875,0,0.465088,0.465088,1,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,27.0,8.6625,0,0.483466,0.483466,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,22.0,12.2875,2,0.418471,0.418471,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,0


In [113]:
train = pd.read_csv('../data/train.csv')
results_train = train['Survived'].copy()
train = train.drop('Survived', axis=1)
train = whole_feature_process(train)

/home/gabriel/anaconda3/envs/Kaggle_Santander/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [114]:
train.head()

,Age,Fare,Relatives,Fare_std,Age_std,Sex_male,Embarked_Q,Embarked_S,binAge_1,binAge_2,...,Pclass_3,Fare_binning_1,Fare_binning_2,Fare_binning_3,Fare_binning_4,Fare_binning_5,Fare_binning_6,Fare_binning_7,Fare_binning_8,Fare_binning_10
0,22.0,7.2500,1,0.502445,0.502445,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,38.0,71.2833,1,0.786845,0.786845,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,26.0,7.9250,0,0.488854,0.488854,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,35.0,53.1000,1,0.420730,0.420730,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
4,35.0,8.0500,0,0.486337,0.486337,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [120]:
train.columns

Index(['Age', 'Fare', 'Relatives', 'Fare_std', 'Age_std', 'Sex_male',
       'Embarked_Q', 'Embarked_S', 'binAge_1', 'binAge_2', 'binAge_3',
       'binAge_4', 'binAge_5', 'binAge_6', 'binAge_7', 'binAge_8', 'binAge_9',
       'binAge_10', 'binAge_11', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'Pclass_2', 'Pclass_3', 'Fare_binning_1', 'Fare_binning_2',
       'Fare_binning_3', 'Fare_binning_4', 'Fare_binning_5', 'Fare_binning_6',
       'Fare_binning_7', 'Fare_binning_8', 'Fare_binning_10'],
      dtype='object')

In [122]:
fm_matrix.columns

Index(['Age', 'Fare', 'Relatives', 'Fare_std', 'Age_std', 'Sex_male',
       'Embarked_Q', 'Embarked_S', 'binAge_1', 'binAge_2', 'binAge_3',
       'binAge_4', 'binAge_5', 'binAge_6', 'binAge_7', 'binAge_8', 'binAge_9',
       'binAge_10', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
       'Pclass_2', 'Pclass_3', 'Fare_binning_1', 'Fare_binning_2',
       'Fare_binning_3', 'Fare_binning_4', 'Fare_binning_5', 'Fare_binning_7',
       'Fare_binning_8', 'Fare_binning_10'],
      dtype='object')

In [118]:
train.shape

(891, 33)

In [119]:
fm_matrix.shape

(418, 32)

In [123]:
fm_matrix['binAge_11'] = 0

In [124]:
from sklearn.naive_bayes import MultinomialNB
modelo = MultinomialNB()
modelo.fit(train, results_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [125]:
previsao = modelo.predict(fm_matrix)

In [131]:
# como eh pra mandar pra eles?
demo = pd.read_csv('../data/gender_submission.csv')

In [133]:
demo.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [135]:
# meus passageiros tao em ordem?
df = pd.read_csv('../data/test.csv')

In [136]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [137]:
# é meio burrice, mas eu quero saber se isso funciona
mask = df['PassengerId'] == demo['PassengerId']

In [138]:
mask.all()

True

In [151]:
submission = df
submission['Survived'] = previsao
submission = submission[['PassengerId', 'Survived']]
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [155]:
submission.to_csv('previsao1.csv', index=False)

In [156]:
oi = pd.read_csv('previsao1.csv')

In [158]:
oi.shape

(418, 2)

# TODO: testar outros modelos, ver como lidar com as variaveis ordinais

# Com forest tree (se tudo der certo)

In [174]:
from sklearn.tree import DecisionTreeClassifier

In [178]:
def choose_depth_forest(depth, treino, treino_marcacoes):
    modelo_forest = DecisionTreeClassifier(max_depth=depth)
    modelo_forest.fit(treino,treino_marcacoes)

    return modelo_forest

In [179]:
treino = train
treino_marcacoes = results_train

In [180]:
depth1 = choose_depth_forest(1, treino, treino_marcacoes)
depth2 = choose_depth_forest(2, treino, treino_marcacoes)
depth3 = choose_depth_forest(3, treino, treino_marcacoes)
depth4 = choose_depth_forest(4, treino, treino_marcacoes)
depth5 = choose_depth_forest(5, treino, treino_marcacoes)

In [184]:
res1 = depth1.predict(fm_matrix)
res2 = depth2.predict(fm_matrix)
res3 = depth3.predict(fm_matrix)
res4 = depth4.predict(fm_matrix)
res5 = depth5.predict(fm_matrix)

In [186]:
def submify_it(res):
    submission = df
    submission['Survived'] = res
    submission = submission[['PassengerId', 'Survived']]
    return submission

In [187]:
res1 = submify_it(res1)
res2 = submify_it(res2)
res3 = submify_it(res3)
res4 = submify_it(res4)
res5 = submify_it(res5)

In [195]:
res1.to_csv('forest_depth1.csv', index=False)
res2.to_csv('forest_depth2.csv', index=False)
res3.to_csv('forest_depth3.csv', index=False)
res4.to_csv('forest_depth4.csv', index=False)
res5.to_csv('forest_depth5.csv', index=False)

# Resultados
> * MultinomialNB = 0.65550 
> * res1 = 0.76076
> * res2 = 0.76555
> * res3 = 0.78947
> * res4 = 0.78468
> * res5 = 0.77033

Done with DecisionTreeClassifier, max_depth = 1, poor feature selection with no regards to ordinal variables

# E agora?

> Melhorar feature selection?

> Ver outros modelos?

> Procurar tecnicas especiais como ensembles e outras coisas que eu nao conheço?